In [12]:
using BenchmarkTools
using LinearAlgebra

## Vecchia funzione

In [13]:
function r(args...)
    args = collect(args)
    n = length(args)
    if n == 1 # rotation in 2D
        angle = args[1]; COS = cos(angle); SIN = sin(angle)
        mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
        mat[1,1] = COS;    mat[1,2] = -SIN;
        mat[2,1] = SIN;    mat[2,2] = COS;
    end

     if n == 3 # rotation in 3D
        mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
        angle = norm(args);
        if norm(args) != 0.0
			axis = args #normalize(args)
			COS = cos(angle); SIN= sin(angle)
			if axis[2]==axis[3]==0.0    # rotation about x
				mat[2,2] = COS;    mat[2,3] = -SIN;
				mat[3,2] = SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[3]==0.0   # rotation about y
				mat[1,1] = COS;    mat[1,3] = SIN;
				mat[3,1] = -SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[2]==0.0    # rotation about z
				mat[1,1] = COS;    mat[1,2] = -SIN;
				mat[2,1] = SIN;    mat[2,2] = COS;
			else
				I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
				Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
				UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
					 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
					 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
				mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
			end
		end
	end
	return mat
end

r (generic function with 1 method)

In [14]:
@btime r(0)

  128.795 ns (2 allocations: 256 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [15]:
@btime r(1,1,1)

  1.160 μs (9 allocations: 1.41 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

## Nuove funzioni

In [16]:
function r2D(args)
	angle = args[1]; COS = cos(angle); SIN = sin(angle)
	mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

r2D (generic function with 1 method)

In [17]:
function rX(mat)
	mat[2,2] = COS;    mat[2,3] = -SIN;
	mat[3,2] = SIN;    mat[3,3] = COS;
	return mat
end

rX (generic function with 1 method)

In [18]:
function rY(mat)
	mat[1,1] = COS;    mat[1,3] = SIN;
	mat[3,1] = -SIN;    mat[3,3] = COS;
	return mat
end

rY (generic function with 1 method)

In [19]:
function rZ(mat)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

rZ (generic function with 1 method)

In [20]:
function rAxis(mat, axis, COS, SIN)
	I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
	Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
	UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
		 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
		 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
	mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
	return mat
end

rAxis (generic function with 1 method)

In [21]:
function r3D(args)
	mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
	angle = norm(args);
	if angle != 0.0
		 axis = args #normalize(args)
		 COS = cos(angle); SIN= sin(angle)
		 if axis[2]==axis[3]==0.0    # rotation about x
			 mat = rX(mat)
		 elseif axis[1]==axis[3]==0.0   # rotation about y
			 mat = rY(mat)
		 elseif axis[1]==axis[2]==0.0    # rotation about z
			 mat = rZ(mat)
		 else
			 mat = rAxis(mat, axis, COS, SIN)
		 end
	 end
	 return mat
end

r3D (generic function with 1 method)

In [22]:
function r(args...)
   args = collect(args)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3D(args)
	end
	return mat
end

r (generic function with 1 method)

In [23]:
@btime r(0)

  187.407 ns (2 allocations: 256 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [24]:
@btime r(1,1,1)

  1.210 μs (9 allocations: 1.41 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [25]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 
  memory estimate:  1.41 KiB
  allocs estimate:  9
  --------------
  minimum time:     1.160 μs (0.00% GC)
  median time:      2.470 μs (0.00% GC)
  mean time:        3.764 μs (1.22% GC)
  maximum time:     1.293 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [26]:
@code_warntype r(1,1,1)

Variables
  #self#::Core.Compiler.Const(r, false)
  args@_2::Tuple{Int64,Int64,Int64}
  n::Int64
  mat::Array{Float64,2}
  args@_5::Union{Tuple{Int64,Int64,Int64}, Array{Int64,1}}

Body::Array{Float64,2}
1 ─      (args@_5 = args@_2)
│        Core.NewvarNode(:(mat))
│        (args@_5 = Main.collect(args@_5::Tuple{Int64,Int64,Int64}))
│        (n = Main.length(args@_5::Array{Int64,1}))
│   %5 = (n == 1)::Bool
└──      goto #3 if not %5
2 ─      (mat = Main.r2D(args@_5::Array{Int64,1}))
3 ┄ %8 = (n == 3)::Bool
└──      goto #5 if not %8
4 ─      (mat = Main.r3D(args@_5::Array{Int64,1}))
5 ┄      return mat


## Modifiche